# Part 1B: Datatypes in HDF5

> Objectives:
> * How to create (and read) HDF5 files with datasets of homogeneous, heterogenous and nested datatypes
> * See how h5py and PyTables achieves the same thing with their own APIs
> * Be introduced to the `IsDescription` class in PyTables for declaring tables (instead of NumPy dtypes)

In [1]:
import numpy as np
import h5py
import tables

In [2]:
import os
import shutil
data_dir = "datatypes"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

## Homogeneous datatypes

In [3]:
arr_to_store = np.arange(10, dtype=np.int8)
arr_to_store

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int8)

### Using h5py

In [4]:
FILENAME = os.path.join(data_dir, "homogenous_h5py.h5")
f = h5py.File(FILENAME, "w")

In [5]:
f.create_dataset(data=arr_to_store, name="mydata")

<HDF5 dataset "mydata": shape (10,), type "|i1">

In [6]:
f['/mydata2'] = arr_to_store    # data can be accessed in a NumPy-like interface

In [7]:
list(f)

['mydata', 'mydata2']

In [8]:
f['/mydata'][:]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int8)

In [9]:
f.close()

In [10]:
!h5ls -rv {FILENAME}

Opened "datatypes\homogenous_h5py.h5" with sec2 driver.
/                        Group
    Location:  1:96
    Links:     1
/mydata                  Dataset {10/10}
    Location:  1:800
    Links:     1
    Storage:   10 logical bytes, 10 allocated bytes, 100.00% utilization
    Type:      native signed char
/mydata2                 Dataset {10/10}
    Location:  1:1400
    Links:     1
    Storage:   10 logical bytes, 10 allocated bytes, 100.00% utilization
    Type:      native signed char


In [11]:
!ls -lh {data_dir}

total 4.0K
-rw-r--r-- 1 tomkooij 197613 2.2K Jun 19 08:31 homogenous_h5py.h5


### Using PyTables

In [12]:
import tables

In [13]:
FILENAME = os.path.join(data_dir, "homogenous_pytables.h5")
f2 = tables.open_file(FILENAME, "w")

In [14]:
f2.create_array(f2.root, name="mydata", obj=arr_to_store)

/mydata (Array(10,)) ''
  atom := Int8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None

In [15]:
f2.root.mydata[:]  # data can be accessed in a NumPy-like interface

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int8)

In [16]:
f2

File(filename=datatypes\homogenous_pytables.h5, title='', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/mydata (Array(10,)) ''
  atom := Int8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None

In [17]:
f2.close()

In [18]:
!h5ls -rv {FILENAME}

Opened "datatypes\homogenous_pytables.h5" with sec2 driver.
/                        Group
    Attribute: CLASS scalar
        Type:      5-byte null-terminated UTF-8 string
        Data:  "GROUP"
    Attribute: PYTABLES_FORMAT_VERSION scalar
        Type:      3-byte null-terminated UTF-8 string
        Data:  "2.1"
    Attribute: TITLE null
        Type:      1-byte null-terminated UTF-8 string

    Attribute: VERSION scalar
        Type:      3-byte null-terminated UTF-8 string
        Data:  "1.0"
    Location:  1:96
    Links:     1
/mydata                  Dataset {10/10}
    Attribute: CLASS scalar
        Type:      5-byte null-terminated UTF-8 string
        Data:  "ARRAY"
    Attribute: FLAVOR scalar
        Type:      5-byte null-terminated UTF-8 string
        Data:  "numpy"
    Attribute: TITLE null
        Type:      1-byte null-terminated UTF-8 string

    Attribute: VERSION scalar
        Type:      3-byte null-terminated UTF-8 string
        Data:  "2.4"
    Location: 

H5tools-DIAG: Error detected in HDF5:tools (1.8.17) thread 0:
  #000: C:\bld\hdf5_1490811383644\work\hdf5-1.8.17\tools\lib\h5tools_dump.c line 1836 in h5tools_dump_mem(): H5Sis_simple failed
    major: Failure in tools library
    minor: error in function
  #001: C:\bld\hdf5_1490811383644\work\hdf5-1.8.17\tools\lib\h5tools_dump.c line 1836 in h5tools_dump_mem(): H5Sis_simple failed
    major: Failure in tools library
    minor: error in function


In [19]:
!ls -lh {data_dir}

total 8.0K
-rw-r--r-- 1 tomkooij 197613 2.2K Jun 19 08:31 homogenous_h5py.h5
-rw-r--r-- 1 tomkooij 197613 2.2K Jun 19 08:31 homogenous_pytables.h5


## Compound Datatypes

In [20]:
dtype = np.dtype([("myfield1", np.int32), ("myfield2", np.float64), ("myfield3", "S5")])
table_to_store = np.fromiter(((i, i**2, "foo_%d"%i) for i in range(10)), dtype=dtype)

In [21]:
table_to_store

array([(0, 0.0, b'foo_0'), (1, 1.0, b'foo_1'), (2, 4.0, b'foo_2'),
       (3, 9.0, b'foo_3'), (4, 16.0, b'foo_4'), (5, 25.0, b'foo_5'),
       (6, 36.0, b'foo_6'), (7, 49.0, b'foo_7'), (8, 64.0, b'foo_8'),
       (9, 81.0, b'foo_9')], 
      dtype=[('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

### Using h5py

In [22]:
FILENAME = os.path.join(data_dir, "compound_h5py.h5")
f = h5py.File(FILENAME, "w")

In [23]:
f['mydata'] = table_to_store

In [24]:
f['mydata']

<HDF5 dataset "mydata": shape (10,), type "|V17">

In [25]:
f['mydata'].dtype

dtype([('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

In [26]:
f['mydata'][:]

array([(0, 0.0, b'foo_0'), (1, 1.0, b'foo_1'), (2, 4.0, b'foo_2'),
       (3, 9.0, b'foo_3'), (4, 16.0, b'foo_4'), (5, 25.0, b'foo_5'),
       (6, 36.0, b'foo_6'), (7, 49.0, b'foo_7'), (8, 64.0, b'foo_8'),
       (9, 81.0, b'foo_9')], 
      dtype=[('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

In [27]:
f.close()

In [28]:
!h5ls -v {FILENAME}

Opened "datatypes\compound_h5py.h5" with sec2 driver.
mydata                   Dataset {10/10}
    Location:  1:800
    Links:     1
    Storage:   170 logical bytes, 170 allocated bytes, 100.00% utilization
    Type:      struct {
                   "myfield1"         +0    native int
                   "myfield2"         +4    native double
                   "myfield3"         +12   5-byte null-padded ASCII string
               } 17 bytes


### Exercise

Open `datatypes\compound_h5py.h5` in PyTables and investigate the dataset.

Look at the dataset description. Read it from disk. Look at the dtype.

In [29]:
FILENAME = os.path.join(data_dir, "compound_h5py.h5")

In [30]:
!ptdump {FILENAME}

/ (RootGroup) ''
/mydata (Table(10,)) ''


In [31]:
# SOLUTION STARTS HERE!!!
fileh = tables.open_file(FILENAME, 'r')

In [32]:
fileh

File(filename=datatypes\compound_h5py.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/mydata (Table(10,)) ''
  description := {
  "myfield1": Int32Col(shape=(), dflt=0, pos=0),
  "myfield2": Float64Col(shape=(), dflt=0.0, pos=1),
  "myfield3": StringCol(itemsize=5, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (3855,)

In [33]:
x = fileh.root.mydata[:]
x

array([(0, 0.0, b'foo_0'), (1, 1.0, b'foo_1'), (2, 4.0, b'foo_2'),
       (3, 9.0, b'foo_3'), (4, 16.0, b'foo_4'), (5, 25.0, b'foo_5'),
       (6, 36.0, b'foo_6'), (7, 49.0, b'foo_7'), (8, 64.0, b'foo_8'),
       (9, 81.0, b'foo_9')], 
      dtype=[('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

In [34]:
x.dtype

dtype([('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

In [35]:
x.shape

(10,)

### Using PyTables (simple way)

In PyTables a compound dataset is called a `Table`.

To store a table we use `create_table`:

In [36]:
FILENAME = os.path.join(data_dir, "compound_pytables1.h5")
f2 = tables.open_file(FILENAME, "w")

In [37]:
table = f2.create_table(f2.root, name="mydata", description=table_to_store.dtype)
table

/mydata (Table(0,)) ''
  description := {
  "myfield1": Int32Col(shape=(), dflt=0, pos=0),
  "myfield2": Float64Col(shape=(), dflt=0.0, pos=1),
  "myfield3": StringCol(itemsize=5, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (3855,)

The `Table` class has high level functions, such as `append()` and `read()`:

In [38]:
table.append(table_to_store)  

In [39]:
table.read()

array([(0, 0.0, b'foo_0'), (1, 1.0, b'foo_1'), (2, 4.0, b'foo_2'),
       (3, 9.0, b'foo_3'), (4, 16.0, b'foo_4'), (5, 25.0, b'foo_5'),
       (6, 36.0, b'foo_6'), (7, 49.0, b'foo_7'), (8, 64.0, b'foo_8'),
       (9, 81.0, b'foo_9')], 
      dtype=[('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

In [40]:
table.remove_row(5)

In [41]:
table.read()

array([(0, 0.0, b'foo_0'), (1, 1.0, b'foo_1'), (2, 4.0, b'foo_2'),
       (3, 9.0, b'foo_3'), (4, 16.0, b'foo_4'), (6, 36.0, b'foo_6'),
       (7, 49.0, b'foo_7'), (8, 64.0, b'foo_8'), (9, 81.0, b'foo_9')], 
      dtype=[('myfield1', '<i4'), ('myfield2', '<f8'), ('myfield3', 'S5')])

In [42]:
f2.close()

### Using PyTables (description way)

In PyTables it is convenient to define compound datasets using the `tables.IsDescription` class, instead of (complicated) numpy dtypes.

In [43]:
class MyTable(tables.IsDescription):
    myfield1 = tables.Int32Col()
    myfield2 = tables.Float64Col()
    myfield3 = tables.StringCol(itemsize=5)

In [44]:
FILENAME = os.path.join(data_dir, "compound_pytables2.h5")
f3 = tables.open_file(FILENAME, "w")

In [45]:
t = f3.create_table(f3.root, "mydata", MyTable)

In [46]:
t.append(table_to_store)

In [47]:
f3.close()

In [48]:
!ls -l {data_dir}

total 156
-rw-r--r-- 1 tomkooij 197613  2314 Jun 19 08:31 compound_h5py.h5
-rw-r--r-- 1 tomkooij 197613 69879 Jun 19 08:31 compound_pytables1.h5
-rw-r--r-- 1 tomkooij 197613 69879 Jun 19 08:31 compound_pytables2.h5
-rw-r--r-- 1 tomkooij 197613  2164 Jun 19 08:31 homogenous_h5py.h5
-rw-r--r-- 1 tomkooij 197613  2154 Jun 19 08:31 homogenous_pytables.h5


Hmm, it seems like PyTables files are larger than h5py ones, why?  Let's introspect a bit into the files:

In [49]:
!h5ls {data_dir}/compound_h5py.h5

mydata                   Dataset {10}


In [50]:
!h5ls {data_dir}/compound_pytables1.h5

mydata                   Dataset {9/Inf}


We see that the dimensionality of the table created with PyTables is `{10/Inf}`, indicating that the dataset is chunked, whereas the one created with h5py is just `{10}`, which means that it is not using chunking.  As chunked datasets take more space than non-chunked ones, this is why PyTables are larger.

The reason why PyTables tables are chunked by default is that they can be enlarged and compressed, and chunking is required in order to allow that.  More on chunking later.

## Nested fields

In [51]:
class NestedTable(tables.IsDescription):
    """A nested table"""
    name = tables.StringCol(10, pos=0)
    
    class momentum(tables.IsDescription):
        p_x = tables.Float64Col()
        p_y = tables.Float64Col()
        p_z = tables.Float64Col() 

In [52]:
FILENAME = os.path.join(data_dir, "nested.h5")
f4 = tables.open_file(FILENAME, "w")

In [53]:
t = f4.create_table(f4.root, "mydata", NestedTable)

In [54]:
t

/mydata (Table(0,)) ''
  description := {
  "name": StringCol(itemsize=10, shape=(), dflt=b'', pos=0),
  "momentum": {
    "p_x": Float64Col(shape=(), dflt=0.0, pos=0),
    "p_y": Float64Col(shape=(), dflt=0.0, pos=1),
    "p_z": Float64Col(shape=(), dflt=0.0, pos=2)}}
  byteorder := 'little'
  chunkshape := (1927,)

In [55]:
dtype = t.dtype
dtype

dtype([('name', 'S10'), ('momentum', [('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])])

In [56]:
table_to_store = np.fromiter((("foo_%s"%i, (i, 10+i, 20+i)) for i in range(10)), dtype=dtype)
table_to_store

array([(b'foo_0', (0.0, 10.0, 20.0)), (b'foo_1', (1.0, 11.0, 21.0)),
       (b'foo_2', (2.0, 12.0, 22.0)), (b'foo_3', (3.0, 13.0, 23.0)),
       (b'foo_4', (4.0, 14.0, 24.0)), (b'foo_5', (5.0, 15.0, 25.0)),
       (b'foo_6', (6.0, 16.0, 26.0)), (b'foo_7', (7.0, 17.0, 27.0)),
       (b'foo_8', (8.0, 18.0, 28.0)), (b'foo_9', (9.0, 19.0, 29.0))], 
      dtype=[('name', 'S10'), ('momentum', [('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])])

In [57]:
table_to_store['momentum']['p_x']

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

In [58]:
t.append(table_to_store)

In [59]:
t.read()

array([(b'foo_0', (0.0, 10.0, 20.0)), (b'foo_1', (1.0, 11.0, 21.0)),
       (b'foo_2', (2.0, 12.0, 22.0)), (b'foo_3', (3.0, 13.0, 23.0)),
       (b'foo_4', (4.0, 14.0, 24.0)), (b'foo_5', (5.0, 15.0, 25.0)),
       (b'foo_6', (6.0, 16.0, 26.0)), (b'foo_7', (7.0, 17.0, 27.0)),
       (b'foo_8', (8.0, 18.0, 28.0)), (b'foo_9', (9.0, 19.0, 29.0))], 
      dtype=[('name', 'S10'), ('momentum', [('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])])

### Using the `Cols` accessor. (PyTables)

`table.col(name)` reads the entire column.

`table.col('momentum')` will read the entire column (array) in memory and slice.

In [60]:
t.col('momentum')[2:5]

array([(2.0, 12.0, 22.0), (3.0, 13.0, 23.0), (4.0, 14.0, 24.0)], 
      dtype=[('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])

Using the `cols` accessor, we can access the column without reading the entire column in memory:

In [61]:
t.cols.momentum

/mydata.cols.momentum (Cols), 3 columns
  p_x (Column(10,), float64)
  p_y (Column(10,), float64)
  p_z (Column(10,), float64)

In [62]:
t.cols.momentum[2:5]

array([(2.0, 12.0, 22.0), (3.0, 13.0, 23.0), (4.0, 14.0, 24.0)], 
      dtype=[('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])

Nested columns can be accessed by the `Cols` accessor using natural naming: 

In [63]:
t.cols.momentum.p_x

/mydata.cols.momentum.p_x (Column(10,), float64, idx=None)

In [64]:
f4.close()

### Exercise

Investigate reading a small part of a large table from disk.

 * Store the table in a HDF5 file. (Using either PyTables or h5py).
 * Read elements [20:30] from the p_x column.

Is the entire datafile being read from disk?

In [65]:
class NestedTable(tables.IsDescription):
    """A nested table"""
    i = tables.Int32Col()
    
    class momentum(tables.IsDescription):
        p_x = tables.Float64Col()
        p_y = tables.Float64Col()
        p_z = tables.Float64Col() 
        
dtype = tables.description.dtype_from_descr(NestedTable)

In [66]:
N = int(1e6)
table_to_store = np.fromiter(((i, (i, i, i)) for i in range(N)), dtype=dtype)

In [67]:
# SOLUTION STARTS HERE
f = tables.open_file('big.h5', 'w')

In [68]:
f.create_table('/', 'mydata', table_to_store)

/mydata (Table(1000000,)) ''
  description := {
  "i": Int32Col(shape=(), dflt=0, pos=0),
  "momentum": {
  "p_x": Float64Col(shape=(), dflt=0.0, pos=0),
  "p_y": Float64Col(shape=(), dflt=0.0, pos=1),
  "p_z": Float64Col(shape=(), dflt=0.0, pos=2)}}
  byteorder := 'little'
  chunkshape := (4681,)

In [69]:
%time f.root.mydata[:]

Wall time: 45 ms


array([(0, (0.0, 0.0, 0.0)), (1, (1.0, 1.0, 1.0)), (2, (2.0, 2.0, 2.0)),
       ..., (999997, (999997.0, 999997.0, 999997.0)),
       (999998, (999998.0, 999998.0, 999998.0)),
       (999999, (999999.0, 999999.0, 999999.0))], 
      dtype=[('i', '<i4'), ('momentum', [('p_x', '<f8'), ('p_y', '<f8'), ('p_z', '<f8')])])

In [70]:
%time f.root.mydata.cols.momentum.p_x[20:30]

Wall time: 3 ms


array([ 20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.])

In [71]:
f.close()

In `h5py` there is no equivalent: 

In [72]:
f = h5py.File('big.h5', 'a')
dset = f['mydata']

In [73]:
%time dset['momentum']['p_x']  # this reads the entire nested columns and selects `p_x`

Wall time: 75 ms


array([  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
         9.99997000e+05,   9.99998000e+05,   9.99999000e+05])

In [74]:
f.close()

# Exercise

Using the hierarchy and compound datasets (tables).

`ufo-scrubbed.csv` is a (scrubbed) partial dataset of UFO Sightings from across the world.

Store the UFO sightings in HDF5. Assume the real (full) dataset is VERY large. Store the data in multiple tables (geospatial).
Make sure you use correct dtype etc.

Use h5py and/or pytables.


In [75]:
import csv

In [76]:
with open('datasets/ufo_scrubbed.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    dataset = [tuple(line) for line in reader]

In [77]:
dataset[:2]

[('datetime',
  'city',
  'state',
  'country',
  'shape',
  'duration (seconds)',
  'duration (hours/min)',
  'comments',
  'date posted',
  'latitude',
  'longitude '),
 ('10/10/1949 20:30',
  'san marcos',
  'tx',
  'us',
  'cylinder',
  '2700',
  '45 minutes',
  'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit',
  '4/27/2004',
  '29.8830556',
  '-97.9411111')]

Let's create a dictonary of sightings per country:

In [78]:
from collections import defaultdict
sightings = defaultdict(list)

for sighting in dataset[1:]:
    dt, city, state, country, _, duration, _, comments, _, lat, lon = sighting
    sightings[country].append((dt, city, state, duration, lat, lon))

In [80]:
sightings.keys()

dict_keys(['us', '', 'gb', 'ca', 'au', 'de'])

In [81]:
len(sightings['de'])

105

In [82]:
#
# SOLUTION STARTS HERE
#

In [83]:
dtype=np.dtype([('datetime', 'S16'), ('city', 'S20'), ('state', 'S2'), ('duration', np.int32), ('lat', 'f8'), ('lon', 'f8')])

In [85]:
np.array(sightings['de'], dtype=dtype)

array([(b'10/13/2006 00:02', b'berlin (germany)', b'', 120, 52.516667, 13.4),
       (b'10/20/2012 18:00', b'berlin (germany)', b'', 1500, 52.516667, 13.4),
       (b'10/8/2012 17:10', b'obernheim (germany)', b'', 2, 49.366667, 7.583333),
       (b'1/10/2011 18:38', b'ottersberg (germany)', b'', 240, 53.1, 9.15),
       (b'11/15/1990 22:30', b'bremen (30 km south ', b'', 30, 50.716667, 10.0),
       (b'11/15/2005 15:00', b'sembach (germany)', b'', 120, 49.516667, 7.85),
       (b'11/18/2002 16:35', b'magdeburg (germany)', b'', 4, 52.166667, 11.666667),
       (b'1/1/2008 22:30', b'neuruppin (germany)', b'', 900, 52.933333, 12.8),
       (b'1/1/2009 00:00', b'lampertheim (germany', b'', 1560, 49.601944, 8.471944),
       (b'1/1/2009 00:15', b'ramstein (germany)', b'', 7200, 49.45, 7.533333),
       (b'12/10/1987 21:00', b'nurenburg (germany)', b'', 28800, 49.447778, 11.068333),
       (b'12/15/1998 22:50', b'senftenberg (germany', b'', 60, 51.516667, 14.016667),
       (b'12/18/2003 20: